Data preparation for feeding the models will be carried out in this file.

In [63]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
from torch.utils.data import Dataset
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [64]:
PATH_OSATS = "../data/raw/OSATS.xlsx"
PATH_TRAIN = "../data/processed/"
BATCH_SIZE = 32

In [65]:
osats = pd.read_excel(PATH_OSATS, engine="openpyxl")
osats

,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE
0,AHO729,E-LEARNING,PRE,1.0,A,P54M,2,1,2,2,1,2,2,1,13
1,AHO729,E-LEARNING,PRE,1.0,B,P54M,2,1,3,1,2,1,2,1,13
2,AHO729,E-LEARNING,PRE,1.0,C,P54M,2,1,1,1,1,1,1,1,9
3,AHO729,E-LEARNING,POST,4.5,A,M45P,4,4,4,3,3,4,3,3,28
4,AHO729,E-LEARNING,POST,4.5,B,M45P,2,3,4,3,3,4,3,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,ZWO108,HMD-BASED,PRE,0.5,B,D94Z,1,1,2,1,1,1,1,1,9
938,ZWO108,HMD-BASED,PRE,0.5,C,D94Z,1,1,1,1,2,1,1,1,9
939,ZWO108,HMD-BASED,POST,3.0,A,Z49D,4,3,3,3,3,3,3,3,25
940,ZWO108,HMD-BASED,POST,3.0,B,Z49D,4,3,4,4,4,5,4,5,33


In [66]:
def grs_to_class(grs):
    if 8 <= grs <= 15:
        return 0  # novice
    elif 16 <= grs <= 23:
        return 1  # intermediate
    elif 24 <= grs <= 31:
        return 2  # proeficient
    elif 32 <= grs <= 40:
        return 3  # specialist
    else:
        return -1  # GRS not in range

osats['GRS'] = osats['GLOBA_RATING_SCORE'].apply(grs_to_class)

osats.head()

,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE,GRS
0,AHO729,E-LEARNING,PRE,1.0,A,P54M,2,1,2,2,1,2,2,1,13,0
1,AHO729,E-LEARNING,PRE,1.0,B,P54M,2,1,3,1,2,1,2,1,13,0
2,AHO729,E-LEARNING,PRE,1.0,C,P54M,2,1,1,1,1,1,1,1,9,0
3,AHO729,E-LEARNING,POST,4.5,A,M45P,4,4,4,3,3,4,3,3,28,2
4,AHO729,E-LEARNING,POST,4.5,B,M45P,2,3,4,3,3,4,3,3,25,2


In [67]:
def processing(image):
    image = np.array(image)

    # Normalization between 0 and 1
    xmax, xmin = image.max(), image.min()
    image = (image - xmin) / (xmax - xmin)

    # C H W (Canal, Altura, Largura)
    image = image.transpose(2, 1, 0)
    return image

In [68]:
def collect_frames(data_path):
    frames = []
    
    for package in os.listdir(data_path):
        package_path = os.path.join(data_path, package)
        
        if os.path.isdir(package_path):
            
            for video in os.listdir(package_path):
                video_path = os.path.join(package_path, video)
                
                if os.path.isdir(video_path):
                    
                    for filename in os.listdir(video_path):
                        if filename.endswith(('.jpg')):
                            img_path = os.path.join(video_path, filename)
                            frames.append(img_path)
    return frames

In [69]:
class GRSDataset(Dataset):
    def __init__(self, path, num_imagens=0, transforms=None):
            
            self.files = collect_frames(path) 
            self.num_imagens = num_imagens if num_imagens > 0 else len(self.files)
            self.transforms = transforms

    def __len__(self):
        return self.num_imagens

    def __getitem__(self, idx):

        img_path = self.files[idx]
        imagem = Image.open(img_path)
        imagem = processing(imagem)
        imagem = imagem.astype(np.float32)

        video_name = os.path.basename(os.path.dirname(img_path))
        label = osats.loc[osats['VIDEO'] == video_name, 'GRS'].values[0]

        if self.transforms:
            imagem = self.transforms(imagem)

        return imagem, label

In [70]:
def prepare_data_loaders(particion, path_train):

    dataset_train = GRSDataset(path_train, transforms=None)

    train_size = int(particion * len(dataset_train))
    val_size = int(0.1 * len(dataset_train))  # 10% for validation
    test_size = len(dataset_train) - train_size - val_size

    train_data, temp_data = random_split(dataset_train, [train_size, len(dataset_train) - train_size], generator=torch.Generator().manual_seed(42))
    val_data, test_data = random_split(temp_data, [val_size, test_size], generator=torch.Generator().manual_seed(42))

    train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

    train_dl_all = DataLoader(train_data, batch_size=len(train_data), shuffle=True)
    val_dl_all = DataLoader(val_data, batch_size=len(val_data), shuffle=True)
    test_dl_all = DataLoader(test_data, batch_size=len(test_data), shuffle=True)

    return train_dl, val_dl, test_dl, train_dl_all, val_dl_all, test_dl_all

In [71]:
train_dl, val_dl, test_dl, train_dl_all, val_dl_all, test_dl_all = prepare_data_loaders(0.70, PATH_TRAIN)

In [72]:
from IPython.display import display

def visualize_data(path):
    # criar uma instância do dataset
    df = pd.read_csv(path, header=0)
    display(df)

def visualize_dataset(train_dl, test_dl, dataset_train, dataset_test):
    print(f"Quantidade de casos de Treino:{len(train_dl.dataset)}")
    print(f"Quantidade de casos de Validação:{len(val_dl.dataset)}")
    print(f"Quantidade de casos de Teste:{len(test_dl.dataset)}")

    x, y = next(iter(train_dl)) # fazer uma iteração nos loaders para ir buscar um batch de casos
    print(f"Shape tensor batch casos treino, input: {x.shape}, output: {y.shape}")
    x, y = next(iter(val_dl)) # fazer uma iteração nos loaders para ir buscar um batch de casos
    print(f"Shape tensor batch casos validação, input: {x.shape}, output: {y.shape}")
    x, y = next(iter(test_dl))
    print(f"Shape tensor batch casos test, input: {x.shape}, output: {y.shape}")

    print(f'Valor maximo:{torch.max(x)} Valor mínimo:{torch.min(x)}')
    x=x.detach().numpy()
    print(f'Valor maximo:{np.max(x)} Valor mínimo:{np.min(x)}')
    print(y)

In [73]:
#visualize_data(PATH_TRAIN)
visualize_dataset(train_dl, test_dl, train_dl_all, test_dl_all)

Quantidade de casos de Treino:21363
Quantidade de casos de Validação:3051
Quantidade de casos de Teste:6105
Shape tensor batch casos treino, input: torch.Size([32, 3, 224, 224]), output: torch.Size([32])
Shape tensor batch casos validação, input: torch.Size([32, 3, 224, 224]), output: torch.Size([32])
Shape tensor batch casos test, input: torch.Size([32, 3, 224, 224]), output: torch.Size([32])
Valor maximo:1.0 Valor mínimo:0.0
Valor maximo:1.0 Valor mínimo:0.0
tensor([1, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2,
        2, 0, 0, 1, 3, 0, 1, 0])
